# Выбор локации для скважины/Choosing a location for a well

Задача: 
добывающей компании нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 
Требуется построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль, проанализировать возможную прибыль и риски техникой *Bootstrap.*

Task:
The mining company needs to decide where to drill a new well.

Oil samples were provided in three regions: in each 10,000 fields, where the quality of oil and the volume of its reserves were measured.
It is required to build a machine learning model that will help determine the region where mining will bring the greatest profit, analyze possible profits and risks using *Bootstrap.* technology

## Загрузка и подготовка данных/Loading and preparing data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import numpy as np
from scipy import stats as st

In [2]:
dataset0 = pd.read_csv('/Users/vladamalkina/Desktop/ЯП проекты/проект 8/geo_data_0_eighth_project.csv')
dataset1 = pd.read_csv('/Users/vladamalkina/Desktop/ЯП проекты/проект 8/geo_data_1_eighth_project.csv')
dataset2 = pd.read_csv('/Users/vladamalkina/Desktop/ЯП проекты/проект 8/geo_data_2_eighth_project.csv')

In [3]:
display(dataset0.head())
display(dataset1.head())
display(dataset2.head())

,Unnamed: 0,id,f0,f1,f2,product
0,0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,2,409Wp,1.022732,0.151990,1.419926,85.265647
3,3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,Unnamed: 0,id,f0,f1,f2,product
0,0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,Unnamed: 0,id,f0,f1,f2,product
0,0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
dataset0.info()
dataset1.info()
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100000 non-null  int64  
 1   id          100000 non-null  object 
 2   f0          100000 non-null  float64
 3   f1          100000 non-null  float64
 4   f2          100000 non-null  float64
 5   product     100000 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory u

Удалили признак, не участвующий в анализе: / Removed a feature that is not involved in the analysis:

In [5]:
dataset0 = dataset0.drop(columns = ['id'])
dataset1 = dataset1.drop(columns = ['id'])
dataset2 = dataset2.drop(columns = ['id'])

## Обучение и проверка модели / Model training and validation

В функции prepare разбили выборки на тренировочную и валидационную. Поскольку в данных присутствуют количественные признаки с разными разбросами значений, нужно привести признаки к одному масштабу: 


In the prepare function, the samples were divided into training and validation. Since the data contains quantitative characteristics with different ranges of values, it is necessary to bring the characteristics to the same scale:

In [6]:
def prepare(data):
    data_train, data_valid = train_test_split(data, test_size = 0.25, random_state = 12345)
    features_train = data_train.drop('product', axis=1)
    target_train = data_train['product']
    features_valid = data_valid.drop('product', axis=1)
    target_valid = data_valid['product']
    numeric = ['f0', 'f1', 'f2']
    pd.options.mode.chained_assignment = None
    scaler = StandardScaler()
    features_train[numeric] = scaler.fit_transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    print(features_train.shape, target_train.shape, features_valid.shape, target_valid.shape)
    return features_train, target_train, features_valid, target_valid

Вызовем функцию prepare для 3 датасетов: 

Let's call the prepare function for 3 datasets:

In [7]:
features0_train = dataset0
features0_train, target0_train, features0_valid, target0_valid = prepare(dataset0)
features1_train, target1_train, features1_valid, target1_valid = prepare(dataset1)
features2_train, target2_train, features2_valid, target2_valid = prepare(dataset2)

(75000, 4) (75000,) (25000, 4) (25000,)
(75000, 4) (75000,) (25000, 4) (25000,)
(75000, 4) (75000,) (25000, 4) (25000,)


In [8]:
def model(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    print('Linear Regression RMSE', mean_squared_error(target_valid, predicted_valid)** 0.5)
    print('средний запас предсказанного сырья', predicted_valid.mean())
    return predicted_valid, predicted_valid.mean()

In [9]:
prediction_0, prediction_0_mean = model(features0_train, target0_train, features0_valid, target0_valid)
prediction_1, prediction_1_mean = model(features1_train, target1_train, features1_valid, target1_valid)
prediction_2, prediction_2_mean = model(features2_train, target2_train, features2_valid, target2_valid)

Linear Regression RMSE 37.57949193854287
средний запас предсказанного сырья 92.5924422447677
Linear Regression RMSE 0.8930659052624607
средний запас предсказанного сырья 68.72849982676497
Linear Regression RMSE 40.03003812272811
средний запас предсказанного сырья 94.96522806981984


prediction_0, prediction_1, prediction_2 - предсказания на валидационной выборке / predictions on the validation set.

prediction_0_mean, prediction_1_mean, prediction_2_mean - средние значения предсказаний на валидационной выборке / average prediction values on the validation set

RMSE модели для 0 региона: / RMSE models for region 0: 37.5794217150813

RMSE модели для 1 региона: / RMSE models for region 1: 0.893099286775617

RMSE модели для 2 региона: / RMSE models for region 2: 40.02970873393434

Средний запас предсказанного сырья для 0 региона: / Average stock of predicted raw materials for region 0: 92.59256778438035

Средний запас предсказанного сырья для 1 региона: / Average stock of predicted raw materials for region 1: 68.728546895446

Средний запас предсказанного сырья для 2 региона: / Average stock of predicted raw materials for region 2: 94.96504596800489

Минимальный размер ошибки модели во 2 регионе. Наиболее точный прогноз модель строит во втором регионе.

The minimum size of the model error in region 2. The model makes the most accurate forecast in the second region.

## Подготовка к расчёту прибыли / Preparation for profit calculation

Средний запас в каждом регионе: / Average stock in each region:

In [10]:
print(dataset0['product'].mean())
print(dataset1['product'].mean())
print(dataset2['product'].mean())

92.49999999999974
68.82500000002561
95.00000000000041


достаточный объём сырья для безубыточной разработки новой скважины:

sufficient volume of raw materials for break-even development of a new well:

In [11]:
min_capacity = 10000000000/(450000*200)
print('достаточный объём сырья для безубыточной разработки новой скважины', min_capacity)

достаточный объём сырья для безубыточной разработки новой скважины 111.11111111111111


Минимальное среднее количество продукта в регионах в 1 скважине / Minimum average amount of product in the regions in 1 well: 111.11111111111111.

Среднее количество продукта в 0 регионе / Average quantity of product in region 0: 92.50000000000001.

Среднее количество продукта в 1 регионе / Average quantity of product in region 1: 68.82500000000002.

Среднее количество продукта во 2 регионе / Average quantity of product in region 2: 95.00000000000004.

Видим, что средние значения в регионах немного меньше, чем минимальное среднее теоретическое. 

We see that the average values ​​in the regions are slightly less than the minimum theoretical average.

Функция для подсчета прибыли для 200 лучших скважин / Function for calculating profits for the 200 best wells:

In [12]:
def revenue(target, probabilities, count):
    cost = 450000
    budget = 10000000000
    target = pd.Series(target, index = target.index)
    probabilities = pd.Series(probabilities, index = target.index)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return cost * selected.sum() - budget

In [13]:
print('для 0 региона:')
print(revenue(target0_valid, prediction_0, 200))
print('для 1 региона:')
print(revenue(target1_valid, prediction_1, 200))
print('для 2 региона:')
print(revenue(target2_valid, prediction_2, 200))

для 0 региона:
3320826043.1398506
для 1 региона:
2415086696.681511
для 2 региона:
2688700460.2221584


## Расчёт прибыли и рисков / Calculation of profits and risks

Распределение прибыли с помощью техники Bootstrap / Profit distribution using Bootstrap technique:

In [14]:
def profit(target, prediction):
    state = np.random.RandomState(12345)
    values = []
    target = pd.Series(target, index = target.index)
    prediction = pd.Series(prediction, index = target.index)
    for i in range(1000):
        target_500 = target.sample(500, replace=True, random_state=state)
        predict_500 = prediction[target_500.index]
        values.append(revenue(target_500, predict_500, 200)) 

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print(values.mean())
    print(lower)
    print(upper)
    print('Риск убытков = {:.2%} '.format((values<0).mean()))

In [15]:
print('Для 0 региона')
profit(target0_valid, prediction_0)
print('Для 1 региона')
profit(target1_valid, prediction_1)
print('Для 2 региона')
profit(target2_valid, prediction_2)

Для 0 региона
425722860.5856449
-102090094.83793654
947976353.3583689
Риск убытков = 6.10% 
Для 1 региона
515228495.8279398
68873225.37050176
931547591.2570494
Риск убытков = 1.00% 
Для 2 региона
435042644.57946986
-128880547.32979088
968640975.2065092
Риск убытков = 6.40% 


Функция для нахождения средней прибыли и  95%-ого доверительного интервала / Function for finding the average profit and 95% confidence interval:

In [16]:
def mean_interval(sample):
    sample = pd.Series(sample*450000)
    print("Cреднее:", sample.mean())
    confidence_interval = st.t.interval(0.95, len(sample)-1, sample.mean(), sample.sem())
    print("95%-ый доверительный интервал:", confidence_interval)

In [17]:
print('Для 0 региона')
mean_interval(prediction_0)
print('Для 1 региона')
mean_interval(prediction_1)
print('Для 2 региона')
mean_interval(prediction_2)

Для 0 региона
Cреднее: 41666599.01014545
95%-ый доверительный интервал: (41537309.49973083, 41795888.52056006)
Для 1 региона
Cреднее: 30927824.922044486
95%-ый доверительный интервал: (30671160.13011207, 31184489.7139769)
Для 2 региона
Cреднее: 42734352.631418966
95%-ый доверительный интервал: (42623599.72246533, 42845105.5403726)


С помощью процедуры Bootstrap вычислили риск убытков / Using the Bootstrap procedure, we calculated the risk of loss:

в 0 регионе / in the 0 region: 6%,

в 1 регионе / in the 1 region: 1%,

во 2 регионе / in the 2 region: 6,4%.

Рекомендованный регион для разработки - 1 регион. / The recommended region for development is 1 region.